In [4]:
import pandas as pd

In [5]:
f = pd.read_csv('../Raw_Datasets/items_Giacomo/items.csv',sep=',')
f['TIPO'].unique()

array(['FUENTE', 'INSTALACIÓN DEPORTIVA', 'PISCINA', 'ZONA VERDE',
       'SENDA', 'CENTRO DE SALUD', 'CICLOCARRIL', 'PASO DE BICI',
       'CARRIL BICI', 'VÍA MIXTA', 'SENDA CICLABLE', 'PISTA BICI',
       'ARCEN BICI', 'ACERA BICI', 'ENCAMINAMIENTO', 'CIRCUITO DE OCIO',
       'BANDA RECOMENDADA', 'GIRO PERMITIDO', 'SENTIDO PERMITIDO'],
      dtype=object)

In [6]:
import json
import csv

# Función para leer datos meteorológicos desde un archivo JSON
def leer_datos_meteorologicos(nombre_archivo):
    with open(nombre_archivo, 'r') as archivo:
        datos = json.load(archivo)
    return datos

# Función para etiquetar la presencia de una ola de calor basada en datos climáticos
def etiquetar_ola_de_calor(datos_clima, umbral_temperatura):
    etiquetas = []
    for registro in datos_clima['data']['list']:
        temperatura = registro['main']['temp']
        if temperatura > umbral_temperatura:
            etiquetas.append(1)  # Hay ola de calor
        else:
            etiquetas.append(0)  # No hay ola de calor
    return etiquetas

# Función para cargar datos de instalaciones desde un archivo CSV y filtrar por tipo de instalación
def cargar_datos_instalaciones(nombre_archivo_csv, tipo_instalacion):
    csv.field_size_limit(500 * 1024)
    datos_instalaciones = []
    with open(nombre_archivo_csv, 'r', encoding='utf-8',newline='') as archivo_csv:
        lector_csv = csv.DictReader(archivo_csv)
        for fila in lector_csv:
            if fila['TIPO'] == tipo_instalacion:
                datos_instalaciones.append(fila)
    return datos_instalaciones

def obtener_recomendaciones(datos_clima, etiquetas_ola_calor, datos_instalaciones, tiene_problemas_salud):
    # Implementa la lógica para generar las recomendaciones de deportes
    # basadas en las condiciones climáticas, la ubicación del usuario y los datos
    # de instalaciones deportivas disponibles.

    recomendaciones = []

    # Implementa la lógica para recomendar deportes seguros según si hay ola de calor o no.
    if etiquetas_ola_calor[0] == 1:  # Primer registro en los datos de etiquetas corresponde a la predicción actual
        # Recomendar actividades en centros de salud si tiene problemas de salud
        if tiene_problemas_salud:
            recomendaciones.extend(obtener_recomendaciones_centros_salud(datos_instalaciones))
    else:
        # Recomendar actividades al aire libre en parques y jardines
        recomendaciones.extend(obtener_recomendaciones_parques(datos_instalaciones))

    return recomendaciones

# Función para obtener recomendaciones específicas para parques y jardines
def obtener_recomendaciones_parques(instalaciones):
    recomendaciones_parques = []
    for instalacion in instalaciones:
        if instalacion['TIPO'] == 'parque':
            recomendaciones_parques.append(f"Hacer ejercicio en el parque: {instalacion['NOMBRE']} ({instalacion['DIRECCION']})")
    return recomendaciones_parques

# Función para obtener recomendaciones específicas para centros de salud
def obtener_recomendaciones_centros_salud(instalaciones):
    recomendaciones_centros_salud = []
    for instalacion in instalaciones:
        if instalacion['TIPO'].lower() == 'centro_salud' and tiene_problemas_salud == True:
            recomendaciones_centros_salud.append(f"Realizar actividad en centro de salud: {instalacion['NOMBRE']} ({instalacion['DIRECCION']})")
    return recomendaciones_centros_salud

if __name__ == "__main__":
    nombre_archivo_json = "../Raw_Datasets/Trabajado/tiempo.json"  # Reemplaza con el nombre de tu archivo JSON
    nombre_archivo_sitios = "../Raw_Datasets/items_Giacomo/items.csv"  # Reemplaza con el nombre de tu archivo CSV de sitios (piscinas, parques y polideportivos)

    umbral_temperatura_ola_calor = 30.0  # Reemplaza con el umbral de temperatura para definir una ola de calor
    tiene_problemas_salud = True  # Reemplaza con True si el usuario tiene problemas de salud, False si no

    datos_clima = leer_datos_meteorologicos(nombre_archivo_json)
    etiquetas_ola_calor = etiquetar_ola_de_calor(datos_clima, umbral_temperatura_ola_calor)

    tipos_instalaciones_deportivas = {'PISCINA', 'ZONA VERDE', 'SENDA', 'CENTRO DE SALUD'}

    datos_instalaciones = []
    for tipo in tipos_instalaciones_deportivas:
        datos_instalaciones.extend(cargar_datos_instalaciones(nombre_archivo_sitios, tipo))

    recomendaciones_deportes = obtener_recomendaciones(
        datos_clima, etiquetas_ola_calor, datos_instalaciones, tiene_problemas_salud
    )

    # Imprime las recomendaciones
    print("Recomendaciones de deportes:")
    for deporte in recomendaciones_deportes:
        print(deporte)


Recomendaciones de deportes:


In [26]:
import json
import csv
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# Función para leer datos meteorológicos desde un archivo JSON
def leer_datos_meteorologicos(nombre_archivo):
    with open(nombre_archivo, 'r') as archivo:
        datos = json.load(archivo)
    return datos

# Función para etiquetar la presencia de una ola de calor basada en datos climáticos
def etiquetar_ola_de_calor(datos_clima, umbral_temperatura):
    etiquetas = []
    for registro in datos_clima['data']['list']:
        temperatura = registro['main']['temp']
        if temperatura > umbral_temperatura:
            etiquetas.append(1)  # Hay ola de calor
        else:
            etiquetas.append(0)  # No hay ola de calor
    return etiquetas

# Función para cargar datos de instalaciones desde un archivo CSV y filtrar por tipo de instalación
def cargar_datos_instalaciones(nombre_archivo_csv, tipo_instalacion):
    csv.field_size_limit(500 * 1024)
    datos_instalaciones = []
    with open(nombre_archivo_csv, 'r', encoding='utf-8', newline='') as archivo_csv:
        lector_csv = csv.DictReader(archivo_csv)
        for fila in lector_csv:
            if fila['TIPO'] == tipo_instalacion:
                datos_instalaciones.append(fila)
    return datos_instalaciones

# Función para obtener recomendaciones de deportes utilizando el modelo de machine learning
def obtener_recomendaciones_deportes_modelo(datos_clima, etiquetas_ola_calor, datos_instalaciones):
    # Obtener los datos históricos de recomendaciones de deportes
    # Aquí asumimos que tienes un archivo CSV llamado 'datos_historicos_deportes.csv'
    # que contiene datos históricos con columnas como 'Temperatura', 'OlaDeCalor', 'TipoInstalacion' y 'Recomendacion'
    df_historico = pd.read_csv('../Raw_Datasets/datos_historicos_deportes.csv')

    # Características (features) del modelo: Temperatura y Ola de calor (1 si hay ola de calor, 0 si no)
    X = df_historico[['', '']].values

    # Etiquetas del modelo: Recomendación de deporte
    y = df_historico['Recomendacion'].values

    # Entrenar el modelo Random Forest
    modelo = RandomForestClassifier(random_state=42)
    modelo.fit(X, y)

    # Obtener los valores actuales de Temperatura y Ola de calor
    temperatura_actual = datos_clima['data']['list'][0]['main']['temp']
    ola_de_calor_actual = etiquetas_ola_calor[0]

    # Realizar la predicción del modelo con los datos actuales
    prediccion_actual = modelo.predict([[temperatura_actual, ola_de_calor_actual]])

    recomendaciones = []

    if prediccion_actual[0] == 1:
        # Si el modelo predice que hay que evitar deportes, entonces:
        # Recomendar actividades en centros de salud si tiene problemas de salud
        if tiene_problemas_salud:
            recomendaciones.extend(obtener_recomendaciones_centros_salud(datos_instalaciones))
    else:
        # Si el modelo predice que se pueden hacer deportes, entonces:
        # Recomendar actividades al aire libre en parques y jardines
        recomendaciones.extend(obtener_recomendaciones_parques(datos_instalaciones))

    return recomendaciones

# Función para obtener recomendaciones específicas para parques y jardines
def obtener_recomendaciones_parques(instalaciones):
    recomendaciones_parques = []
    for instalacion in instalaciones:
        if instalacion['TIPO'] == 'parque':
            recomendaciones_parques.append(f"Hacer ejercicio en el parque: {instalacion['NOMBRE']} ({instalacion['DIRECCION']})")
    return recomendaciones_parques

# Función para obtener recomendaciones específicas para centros de salud
def obtener_recomendaciones_centros_salud(instalaciones):
    recomendaciones_centros_salud = []
    for instalacion in instalaciones:
        if instalacion['TIPO'].lower() == 'centro_salud' and tiene_problemas_salud:
            recomendaciones_centros_salud.append(f"Realizar actividad en centro de salud: {instalacion['NOMBRE']} ({instalacion['DIRECCION']})")
    return recomendaciones_centros_salud

if __name__ == "__main__":
    nombre_archivo_json = "../Raw_Datasets/Trabajado/tiempo.json"  # Reemplaza con el nombre de tu archivo JSON
    nombre_archivo_sitios = "../Raw_Datasets/items_Giacomo/items.csv"  # Reemplaza con el nombre de tu archivo CSV de sitios (piscinas, parques y polideportivos)

    umbral_temperatura_ola_calor = 30.0  # Reemplaza con el umbral de temperatura para definir una ola de calor
    tiene_problemas_salud = True  # Reemplaza con True si el usuario tiene problemas de salud, False si no

    datos_clima = leer_datos_meteorologicos(nombre_archivo_json)
    etiquetas_ola_calor = etiquetar_ola_de_calor(datos_clima, umbral_temperatura_ola_calor)

    tipos_instalaciones_deportivas = {'PISCINA', 'ZONA VERDE', 'SENDA', 'CENTRO DE SALUD'}

    datos_instalaciones = []
    for tipo in tipos_instalaciones_deportivas:
        datos_instalaciones.extend(cargar_datos_instalaciones(nombre_archivo_sitios, tipo))

    # Obtener recomendaciones de deportes utilizando el modelo de machine learning
    recomendaciones_deportes_modelo = obtener_recomendaciones_deportes_modelo(
        datos_clima, etiquetas_ola_calor, datos_instalaciones
    )

    # Imprime las recomendaciones del modelo
    print("Recomendaciones de deportes (modelo):")
    for deporte in recomendaciones_deportes_modelo:
        print(deporte)


KeyError: "None of [Index(['Temperatura', 'OlaDeCalor'], dtype='object')] are in the [columns]"

In [11]:
X = pd.read_csv('../Raw_Datasets/items_Giacomo/deportes_final.csv',sep=',')
X['Actividad'].unique()

array(['BMX', 'Ciclismo', 'Ciclismo estacionario', 'Calistenia',
       'Entrenamiento en circuito', 'Levantamiento de peso',
       'Máquina escaladora', 'Aeróbicos', 'Yoga', 'Aeróbicos acuáticos',
       'Correr', 'Atletismo', 'Bádminton', 'Baloncesto', 'Bolos',
       'Billar', 'Boxeo', 'Dardos', 'Frisbee', 'Ultimate frisbee', 'Golf',
       'Minigolf', 'Gimnasia', 'Balonmano', 'Hockey', 'Petanca', 'Calva',
       'Artes marciales', 'Kickboxing', 'Kickball', 'Pádel', 'Raquetbol',
       'Escalada', 'Salto a la comba', 'Chito', 'Skateboard', 'Patinaje',
       'Fútbol', 'Fútbol sala', 'Sóftbol', 'Béisbol',
       'Tenis de mesa / ping pong', 'Tai chi', 'Tenis', 'Tenis en pareja',
       'Voleibol', 'Marcha rápida', 'Montañismo', 'Caminar', 'Natación',
       'Nado sincronizado', 'Waterpolo', 'Voleibol acuático',
       'Patinaje sobre hielo', 'Frontenis', 'Senderismo'], dtype=object)

In [23]:
import pandas as pd
import numpy as np
import random
import string
from datetime import datetime, timedelta

# Lista de deportes
deportes = np.array(['BMX', 'Ciclismo', 'Ciclismo estacionario', 'Calistenia', 'Entrenamiento en circuito',
                     'Levantamiento de peso', 'Máquina escaladora', 'Aeróbicos', 'Yoga', 'Aeróbicos acuáticos',
                     'Correr', 'Atletismo', 'Bádminton', 'Baloncesto', 'Bolos', 'Billar', 'Boxeo', 'Dardos',
                     'Frisbee', 'Ultimate frisbee', 'Golf', 'Minigolf', 'Gimnasia', 'Balonmano', 'Hockey',
                     'Petanca', 'Calva', 'Artes marciales', 'Kickboxing', 'Kickball', 'Pádel', 'Raquetbol',
                     'Escalada', 'Salto a la comba', 'Chito', 'Skateboard', 'Patinaje', 'Fútbol', 'Fútbol sala',
                     'Sóftbol', 'Béisbol', 'Tenis de mesa / ping pong', 'Tai chi', 'Tenis', 'Tenis en pareja',
                     'Voleibol', 'Marcha rápida', 'Montañismo', 'Caminar', 'Natación', 'Nado sincronizado',
                     'Waterpolo', 'Voleibol acuático', 'Patinaje sobre hielo', 'Frontenis', 'Senderismo'])

# Generar un DataFrame sintético con datos históricos de recomendaciones de deportes
np.random.seed(42)
num_datos = 1000

# Simular datos de Fecha y Hora
fechas = [datetime.now() - timedelta(days=i) for i in range(num_datos)]
horas = [random.randint(0, 23) for _ in range(num_datos)]

# Simular datos de Clima
clima = np.random.choice(['Bueno', 'Malo', 'Lluvioso'], num_datos)

# Simular datos de Temperatura en el rango de 10 a 35 grados Celsius
temperatura = np.random.uniform(10, 35, num_datos)

# Simular datos de Humedad en el rango de 30 a 80 por ciento
humedad = np.random.randint(30, 81, num_datos)

# Simular datos de Velocidad del Viento en el rango de 0 a 20 km/h
velocidad_viento = np.random.randint(0, 21, num_datos)

# Simular datos de Edad en el rango de 18 a 65 años
edad = np.random.randint(18, 66, num_datos)

# Simular deportes que ha hecho antes
deporte_practicado = np.random.choice(deportes, num_datos)

# Simular datos de distancia en el rango de 0.5 a 20 kilómetros
distancia = np.random.uniform(0.5, 20, num_datos)

# Simular datos de peso en el rango de 50 a 100 kilogramos
peso = np.random.uniform(50, 100, num_datos)

# Simular objetivos calóricos
objetivos_caloricos = np.random.choice(['Bajo', 'Medio', 'Alto'], num_datos)

calorias_gastadas = (distancia * 0.5 * peso).round(2)

# Simular condición física
condicion_fisica = np.random.choice(['Malo', 'Bueno', 'En Forma'], num_datos)

# Generar nombres aleatorios para cada registro
def generar_nombre_aleatorio():
    return ''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase) for _ in range(6))

nombres_aleatorios = [generar_nombre_aleatorio() for _ in range(num_datos)]

# Generar nombres de personas para cada registro
# nombres_personas = np.random.choice(['John', 'Emily', 'Michael', 'Jessica', 'William', 'Sophia', 'Christopher', 'Emma', 'Daniel', 'Olivia'], num_datos)

Genero = np.random.choice([0,1])

# Crear el DataFrame
df_historico = pd.DataFrame({
    'Fecha': [fecha.date() for fecha in fechas],
    'Hora': horas,
    # 'Nombres': nombres_personas,
    'Genero': Genero,
    'Clima': clima,
    'Temperatura (°C)': temperatura.round().astype(int),
    'Humedad': humedad,
    'Edad': edad,
    'DeportePracticado': deporte_practicado,
    'Distancia (Km)': distancia.round(2),
    'Peso (Kg)': peso.round(2),
    'ObjetivosCaloricos': objetivos_caloricos,
    'CaloriasGastadas': calorias_gastadas,
    'CondicionFisica': condicion_fisica,
    
})

# Convertir columna de condición física en variables dummy
# condicion_fisica_dummies = pd.get_dummies(df_historico['CondicionFisica'], prefix='Condicion')
# df_historico = pd.concat([df_historico, condicion_fisica_dummies], axis=1)

# Convertir columna de objetivos calóricos en variables dummy
# objetivos_caloricos_dummies = pd.get_dummies(objetivos_caloricos, prefix='Objetivo')
# df_historico = pd.concat([df_historico, objetivos_caloricos_dummies], axis=1)

# Convertir columna de deporte practicado en variables dummy y multiplicar por las calorías gastadas en cada deporte
deporte_practicado_dummies = pd.get_dummies(df_historico['DeportePracticado'], prefix='Deporte')
for deporte in deportes:
    df_historico['Deporte_' + deporte] = deporte_practicado_dummies['Deporte_' + deporte] * df_historico['CaloriasGastadas'] * df_historico['CaloriasGastadas']



# Seleccionar solo las columnas numéricas para aplicar el redondeo
columnas_numericas = df_historico.select_dtypes(include=[np.number]).columns

# Redondear solo las columnas numéricas a los primeros 6 dígitos
df_historico[columnas_numericas] = df_historico[columnas_numericas].round(0)






# Guardar el DataFrame en un archivo CSV
df_historico.to_csv('../Raw_Datasets/datos_historicos_deportes.csv', index=False)


In [17]:
df_historico.iloc[995].head(10)

Fecha                 2020-11-06
Hora                          17
Nombres              Christopher
Clima                       Malo
Temperatura (°C)              33
Humedad                       46
Edad                          38
DeportePracticado      Waterpolo
Distancia (Km)               2.0
Peso (Kg)                   99.0
Name: 995, dtype: object

In [24]:
df_historico

,Fecha,Hora,Genero,Clima,Temperatura (°C),Humedad,Edad,DeportePracticado,Distancia (Km),Peso (Kg),...,Deporte_Marcha rápida,Deporte_Montañismo,Deporte_Caminar,Deporte_Natación,Deporte_Nado sincronizado,Deporte_Waterpolo,Deporte_Voleibol acuático,Deporte_Patinaje sobre hielo,Deporte_Frontenis,Deporte_Senderismo
0,2023-07-29,10,0,Lluvioso,12,59,23,Waterpolo,3.0,70.0,...,0.0,0.0,0.0,0.0,0.0,10758.0,0.0,0.0,0.0,0.0
1,2023-07-28,3,0,Bueno,23,43,58,Bolos,4.0,63.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2023-07-27,7,0,Lluvioso,20,35,53,Voleibol,9.0,67.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2023-07-26,7,0,Lluvioso,35,32,19,Hockey,15.0,90.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2023-07-25,18,0,Bueno,13,45,46,Golf,18.0,81.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2020-11-06,5,0,Malo,33,46,38,Waterpolo,2.0,99.0,...,0.0,0.0,0.0,0.0,0.0,9303.0,0.0,0.0,0.0,0.0
996,2020-11-05,13,0,Malo,13,53,50,Fútbol sala,3.0,69.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
997,2020-11-04,22,0,Lluvioso,30,50,55,Voleibol acuático,17.0,63.0,...,0.0,0.0,0.0,0.0,0.0,0.0,305942.0,0.0,0.0,0.0
998,2020-11-03,6,0,Lluvioso,20,80,32,Tenis,10.0,71.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
